In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import warnings
from tabulate import tabulate
import xgboost as xgb
import dask.dataframe as dd
warnings.filterwarnings("ignore")

In [2]:
iv_features = pd.read_csv("../reports/iv_features.csv")

In [21]:
# train.drop('WOE_target',axis=1, inplace=True)
features = iv_features["useful"].to_list()
features.remove("target")
len(features)
features = features[:-4]

In [4]:
#First slice
features = features[:50]
features.insert(0, "customer_ID")

In [4]:
#Second slice
features = features[50:]
features.insert(0, "customer_ID")

In [4]:
train = pd.read_parquet("../data/processed/train_withlabels.parquet")

In [5]:
test = pd.read_parquet("../data/raw/test.parquet")

In [6]:
train = train.groupby('customer_ID').tail(1).set_index('customer_ID')
test = test.groupby('customer_ID').tail(1).set_index('customer_ID')

In [23]:
test=test[features]

In [ ]:
# # Features for balancing
# features = train.columns.to_list()
# features = features[1:68]

## Preparing test set for submission

In [24]:

def iv_woe(data, feature, target,bins=10, show_woe=False, show_iv= False, balance= True):
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    #Extract Column Names
    cols = [feature]
    #Run WOE and IV on all the independent variables
    for ivars in cols:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop', precision=7)
            data[ivars]= binned_x
            data[ivars] = data[ivars].astype(str)
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
            d0 = d0.astype({"x": str})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
            data[ivars] = data[ivars]
    
    d = d0.groupby("x", as_index=False, dropna=False).agg({"y": ["count", "sum"]})
    d.columns = ['Cutoff', 'N', 'Events']
    
    d.insert(loc=0, column='Variable', value=ivars)

    d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()
    d['Non-Events'] = d['N'] - d['Events']
    d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()
    d['WoE'] = np.log(d['% of Non-Events']/d['% of Events'])
    d['IV'] = d['WoE'] * (d['% of Non-Events']-d['% of Events'])
    
    if (d["Cutoff"].dtypes == "object"):
            d.replace({"nan":" NaN,NaN "}, inplace=True)
            d["min"] = d["Cutoff"].apply(lambda x: x.split(',')[0][1:7])
            d["max"] = d["Cutoff"].apply(lambda x: x.split(',')[1][:-1])
            d.replace({"NaN":np.NaN}, inplace=True)
            d["min"] = d["min"].astype("float")
            d["max"] = d["max"].astype("float")
    else:
        d["min"] = d["Cutoff"].apply(lambda x: x)
        d['max'] = d["Cutoff"].apply(lambda x: x)
        
    temp = pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
    newDF=pd.concat([newDF,temp], axis=0)
    woeDF=pd.concat([woeDF,d], axis=0)
    if balance:
        balance_col = ["WoE","min", "max"]
        woeDF = woeDF[balance_col]
        return woeDF
    return newDF, woeDF

In [25]:
variaveis = test[features].columns
#df1 = {}
l = 0
for col in variaveis:
    if col == "target" or col == "customer_ID" or col == "S_2": continue
    else:
        print("WoE e IV: {}".format(col))
        df = iv_woe(train, col, "target",)
        # df1[l] = df
        # df1[l]["nome"] = "WOE_"+col
        print(l)
        l = l + 1
        #print(tabulate(df, headers="keys"))
        #print("IV score: {:.2f}".format(iv))
        print("\n")
        i = 0
        for i in range(0,len(df)):
            if df[i:i+1]["min"].isna().values:
                test.loc[test[col].isna(), col] = df.iloc[i,0]
            else:
                test.loc[test[col].between(df.iloc[i,1], df.iloc[i,2], inclusive=True), col] =  df.iloc[i,0]
        # del df
        gc.collect()



WoE e IV: D_39
0


WoE e IV: D_42
1


WoE e IV: B_5
2


WoE e IV: D_46
3


WoE e IV: D_49
4


WoE e IV: B_8
5


WoE e IV: D_50
6


WoE e IV: R_3
7


WoE e IV: S_5
8


WoE e IV: S_6
9


WoE e IV: B_12
10


WoE e IV: S_8
11


WoE e IV: D_56
12


WoE e IV: B_13
13


WoE e IV: D_59
14


WoE e IV: S_11
15


WoE e IV: D_63
16


WoE e IV: D_64
17


WoE e IV: D_68
18


WoE e IV: S_12
19


WoE e IV: R_6
20


WoE e IV: S_13
21


WoE e IV: B_21
22


WoE e IV: D_69
23


WoE e IV: D_71
24


WoE e IV: D_72
25


WoE e IV: S_15
26


WoE e IV: P_4
27


WoE e IV: D_76
28


WoE e IV: B_24
29


WoE e IV: B_26
30


WoE e IV: D_79
31


WoE e IV: R_9
32


WoE e IV: S_16
33


WoE e IV: D_81
34


WoE e IV: D_83
35


WoE e IV: R_15
36


WoE e IV: D_86
37


WoE e IV: D_88
38


WoE e IV: B_31
39


WoE e IV: R_19
40


WoE e IV: B_32
41


WoE e IV: S_20
42


WoE e IV: R_21
43


WoE e IV: D_89
44


WoE e IV: R_22
45


WoE e IV: D_91
46


WoE e IV: D_92
47


WoE e IV: D_94
48


WoE e IV: R_24
49


WoE e IV: R_25
50



_____________________________________________

In [28]:
test.to_parquet("../data/final/test.parquet")

In [29]:
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [29]:
test = pd.read_parquet("../data/final/test.parquet")
len(test.columns)

103

In [30]:
test = test.groupby("customer_ID").tail(1).set_index("customer_ID")


In [31]:
X_test = test[features]
del test
dtest = xgb.DMatrix(data = X_test)
del X_test

In [32]:
xgb_model = xgb.Booster()
xgb_model.load_model('../models/XGB_V_fold4.xgb')

In [33]:
xgb_test_preds = []

xgb_test_preds.append(xgb_model.predict(dtest))

In [34]:
sub['prediction']=np.mean(xgb_test_preds, axis=0)
sub.to_csv('../data/final/submission.csv', index=False)
display(sub.head())

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.879368
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.547567
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.984291
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.982672
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.996247
